# Creating and Publishing a Python Library

Early in the course we showed how easy it is to download packages from PyPI (Python Package Index), the official repository for all Python pacakges. This class will discuss process of creating and publishing a library to the outer world, which is a bit more complicated than downloading a package, but not by much.

## Naive Solution

The "dumbest" way to do it is to send a `.py` file to someone by mail. If the receiver has a Python interpreter installed, he can just run the file by writing `python new_file.py` in his command line. This has a few short-comings:

1. The script was written in a specific Python version. You have to make sure that your receiver has the same Python version.
2. If it's more than one script, you have to bundle scripts together and make sure they remain in the same directory when run.
3. Nearly all scripts have dependencies. There's no way to verify that the receiver truly has all the dependencies and is using the same version of dependencies that you're using.
4. If the original script is using code not written in Python, like Cython code or other C\C++ modules, you need to build these scripts in place - you can't rely on them working on the target computer.

These issues, along with the relative ease with which one can publish Python code online, support a more robust approach to code sharing.

## Python Libraries from the Ground Up

There are a few "levels" for Python projects and libraries. This tutorial will guide you through the process of creating a full-blown project, including PyPI publishing and test-suite support. If you're aiming at something lighter you can just drop some of the components used here.

Features of this project include:
1. GitHub integration
2. Automatic testing via TravisCI
3. Semantic versioning support
4. Integration of C source files
5. Easy PyPI uploading
6. Automatic docs generation.

Most of these features rely on a good `cookiecutter` template, so let's start with that.

### Cookiecutter

Happily, other people already went through the trouble of publishing a Python library, and a few of them automated this process using a tool called `cookiecutter`, a Python library to create other Python libraries.

1. The first step is to create a new environment, named `py36` and install `cookiecutter`:

    `conda create -n py36 python=3.6 cookiecutter`
    
    `source activate py36`

2. Navigate in your command line to the folder in which you want your project folder to be in. For example, if I wish to create a new project, named `parse_stuff` in `/home/hagai/parse_stuff`, I need to navigate to `/home/hagai`.

3. Generate your project with: `cookiecutter https://github.com/audreyr/cookiecutter-pypackage.git`. It will start asking questions regarding the project. The `project_name` question should be answered with `parse_stuff`, in our case. In square brackets you'll see an example of what it expects. 
    - When it prompts you `use_pytest` write `y`.
    - When it prompts you `use_pypi_deployment_with_travis` write `y`.
    - If you wish to have a command line interface (CLI) to your program you can tell it so. Else, choose no.

4. After choosing a license it will generate a new folder with the `project_name` name and a whole bunch of files and folders in it. We'll discuss most of them later on, but for now you should note that your actual code goes into the folder named like your project (it will already have an `__init__.py` file waiting, as well as a `parse_stuff.py` "main" file). Also, your tests go, naturally, into the `tests` folder.

### git and GitHub

Now you should create a GitHub repo for this project. Even for small, "insignificant" projects, having version control and an online backup is always a good idea.

1. Go to your GitHub account and create a new project with a name matching the `project_slug` variable (the name of the folder in your file system). You don't need a license or a .gitignore file, an empty project will do just fine.

2. Copy the link given at the top of the page (`https://.../parse_stuff.git`).

3. Now we'll setup `git` locally. Write the following in the terminal (you need to install `git` first for it to work):
    ```bash
    cd parse_stuff
    git init .
    git add .
    git commit -m "Initial skeleton"
    git remote add origin URL_TO_REPO.
    git push origin master
    ```

### Setting a `dev` environment

Even though the project doesn't really exist yet, in order to use some of its features, like automated testing, you have to install several libraries. Do that by installing the packages listed in `requirements_dev.txt`:

`pip install -r requirements_dev.txt`

The two important packages that will be installed here are `bumpversion` and `twine`.

### Setting Up TravisCI

Travis is used to automatically test your code whenever you push something to GitHub. It works by pulling the latest commit from your GitHub into its servers. It then builds an environment identical to the one you have in your computer, installs your package and runs the tests for that package. Lastly it generates a detailed error report if one or more of your tests failed.

It's completely free and pretty easy to use. Create an account there using your GitHub credentials. When you see the list of repos you have, switch the new repo to "on" in the box right next to it.

Tests are run in TravisCI with `tox`, another small Python library that automates testing. `tox` automatically comes with support for Python 2.7, 3.4, 3.5 and 3.6, but that's a bit of an overkill in our case. In principal, writing `tox` in the command line should run all tests for your code, but if you do it now it will probably fail since it doesn't recognize all of these environments. We'll skip it for now.

Make a trivial change somewhere in the codebase, for example add `print("Hello, world!")` in a Python file. Add, commit, and push this change to your repo. Check TravisCI's status and verify that it built and ran your project's tests correctly. Since the tests are empty, they should pass, giving your a green status in Travis.

### ReadTheDocs

ReadTheDocs is a website that happily hosts the documentation for your code. It will automatically fetch whatever's in your `docs` folder and display it in a nice-looking website.

1. Create an account [there](https://readthedocs.org) and log into it.

2. Go to your dashboard and Import a repository from the "My Projects" pulldown.

3. Inside GitHub, select Settings -> Webhooks and turn on the ReadTheDocs hook.

Your docs will update online every time you update them and push the changes to GitHub.

### `pyup.io`

This service makes sure all your requirements are up-to-date. It will send you a pull request whenever a new release is available for one of your dependencies.

1. Create an account in [pyup.io](https://pyup.io) and log into it.

2. Click on "Add Repo" and "Pin" your dependencies.

### Setting Up a PyPI Release

Finally, we're ready to release our package so it would be `pip`-installable. First, though, we have to setup all pieces in place.

1. Create two accounts for the ["Live"](https://pypi.org/account/register/) and ["Test"](https://test.pypi.org/account/register/) versions of PyPI. It's helpful to use the same username and password for both sites.

2. Create a `.pypirc` file in your home folder (`C:\Users\MyName` in Windows, `~` in Mac\Unix.) This file will contain sensitive information on your accounts, so don't upload it anywhere! The file should contain the following:

    ```ini
    [distutils]
    index-servers =
      pypi
      pypitest

    [pypi]
    repository=https://pypi.python.org/pypi
    username=your_username
    password=your_password

    [pypitest]
    repository=https://testpypi.python.org/pypi
    username=your_username
    password=your_password
    ```

3. Assert that `bumpversion` was downloaded to allow for easier control of our version number. Most packages use Semantic Versioning, which means that each version number has 3 digits:
    ```
      0   .   1   .   9
    major   minor   patch
    ```
    __Major version__ updates have breaking changes to the core functionality of the app, like the transition from Python 2 to 3. 
    
    __Minor version__ updates add functionality to the app without breaking existing functionality. For example, the addition of f-strings to Python 3.6.
    
    __Patches__ fix the app without introducing any API changes.
    
    To use `bumpversion` you simply write (in the home folder of the project) `bumpversion patch\minor\major`, in accordance to the patch you're applying.

### Uploading to PyPI

Follow these instructions whenever you wish to upload a package to PyPI.
1. Remove the contents of the `build` folder.
2. Use `bumpversion` to bump the version according to the patch you're applying.
3. Run tests: Type `tox`.
4. Commit your new changes:
    ```bash
    git add .
    git commit -m "My new patch"
    git push
    ```
1. Build a source distribution of your package by typing:
`python setup.py clean --all sdist bdist_wheel`
4. Release to PyPI:
    `twine upload --skip-existing dist/*`